In [1]:
if "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime
import gc
import pandas as pd
import joypy

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.datasets as datasets

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import timedelta

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/dist

In [3]:
# PCA_ = PCA(n_components=60).fit(X_norm)
# X_PCA = PCA_.transform(X_norm)
# # X_TSNE = TSNE(n_components=60, learning_rate="auto", init='random', random_state=0).fit_transform(X_PCA)
# reducer = umap.UMAP(min_dist=0.9, n_components=60)
# X_UMAP = reducer.fit_transform(X_norm)
# X_batch = np.reshape(X_UMAP, (X_UMAP.shape[0]//5,5,X_UMAP.shape[1]))
# # X_UMAP_Norm = Normalizer().fit_transform(X_UMAP)

In [5]:
root_audios_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv"
root_recorders_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_recorders.csv"

audios = pd.read_csv(root_audios_Zamuro, index_col=0)
recorders = pd.read_csv(root_recorders_Zamuro, index_col=0)

def combinar_nombre_ubicacion(row):
    return f"{row['field_number_PR']}_{row['Filename']}"

# Aplicando la función a cada fila del DataFrame para crear la nueva columna
audios['Filename_'] = audios.apply(combinar_nombre_ubicacion, axis=1)

audios.set_index("Filename_", inplace=True)

# AE Model

In [14]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

models = [10,20,30,40,50]

accuracies = []
f1_scores = []
recalls = []
for  i, mod_id in enumerate(models):

    X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/epochs_50/{mod_id}/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
    X_ae = np.asarray(X_ae)
    y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/epochs_50/{mod_id}/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))
    
    y = np.asarray(y_path)
    y_2 = y[:,0]
    for i in range(len(y_2)):
        y_2[i] = y_2[i][0:-2] 
    y_2 = list(y_2)
    y_3 = np.repeat(y_2, 5)
    labels_ae = []
    for i in range(len(y_3)):
        labels_ae.append(audios.loc[y_3[i], "cover"])
        
    X_train, X_test, y_train, y_test = train_test_split(X_ae, labels_ae, test_size=0.2,random_state=0)
    
    clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
    clf_rf.fit(X_train, y_train)

    y_pred_rf = clf_rf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred_rf)
    f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
    recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
    print("Accuracy:", accuracy)
    print("f1:", f1_score)
    print("recall", recall)
    
    accuracies.append(accuracy)
    f1_scores.append(f1_score)
    recalls.append(recall)
    
np.save(f"{root}/Zamuro/temporal_zamuro/accuracies.npy", accuracies)
np.save(f"{root}/Zamuro/temporal_zamuro/f1_scores.npy", f1_scores)
np.save(f"{root}/Zamuro/temporal_zamuro/recalls.npy", recalls)

# X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
# X_ae = np.asarray(X_ae)
# y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))

# scaler = StandardScaler().fit(X_ae)
# X_scaled = scaler.transform(X_ae)

# Normalizer_ = Normalizer().fit(X_ae)
# X_norm = Normalizer_.transform(X_ae)


Accuracy: 0.789130702077954
f1: 0.7176879443843872
recall 0.6920133258783393
Accuracy: 0.7917896225255133
f1: 0.7212599325770599
recall 0.695738104162987
Accuracy: 0.79080597565474
f1: 0.719859892569811
recall 0.6945857688114967
Accuracy: 0.7899452846428132
f1: 0.7181349504855311
recall 0.6927348125279907
Accuracy: 0.7899145456781016
f1: 0.7188050972016402
recall 0.6935557365932045


In [6]:
X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
X_ae = np.asarray(X_ae)

y = np.asarray(y_path)
y_2 = y[:,0]
for i in range(len(y_2)):
    y_2[i] = y_2[i][0:-2] 
y_2 = list(y_2)
y_3 = np.repeat(y_2, 5)
labels_ae = []
for i in range(len(y_3)):
    labels_ae.append(audios.loc[y_3[i], "cover"])
    
X_ae_unflat = np.reshape(X_ae, [X_ae.shape[0]//5,5,X_ae.shape[1]])
X_df = pd.DataFrame(X_ae)
X_df["y"] = y_3

In [69]:
def extract_parts(row):
    parts = row.split('_')
    location = parts[0]
    date = parts[1]
    time = parts[2].split('.')[0]  # Eliminar la extensión .WAV
    day = date[-2:]  # Últimos dos caracteres para el día
    hour = time[:2]
    return pd.Series([location, day, hour])

# Aplicar la función a la columna 'y' y crear nuevas columnas
X_df[['location', 'day', 'hour']] =X_df['y'].apply(extract_parts)

def define_hour_stage(hour):
    hour = int(hour)
    if 5 <= hour <= 8:
        return 'morning'
    elif 9 <= hour <= 16:
        return 'day'
    else:
        return 'night'

X_df['hour_stage'] =X_df['hour'].apply(define_hour_stage)

In [94]:
df_day={}
for i in ["04","05","06","07","08","09","10","11","12","13"]:
    df_day = X_df[X_df['day'].isin([i])]
    df_day.reset_index(inplace=True)
    print(len(df_day))

25180
39170
43500
43640
38525
30515
33700
35615
22140
6705


In [93]:
X_ae.nbytes

6745835520

In [67]:
X_ae[234564], y_3[234564]

(array([26.839605 , 27.068113 , 26.859838 , ...,  6.802875 ,  7.1015186,
         7.026036 ], dtype=float32),
 'RZUF11_20220911_193000.WAV')

In [64]:
X_df[234564:234584]

,0,1,2,3,4,5,6,7,8,9,...,5175,5176,5177,5178,5179,5180,5181,5182,5183,y
234564,26.839605,27.068113,26.859838,26.522417,26.644791,26.807615,27.375324,28.793173,27.634897,26.052017,...,6.914966,6.748425,6.946465,6.673808,6.941545,6.755345,6.802875,7.101519,7.026036,RZUF11_20220911_193000.WAV
234565,30.813412,32.515778,32.588451,34.133400,35.407177,34.443405,30.942329,31.136604,34.383751,26.456844,...,7.436952,7.805335,6.836241,6.803342,7.300022,6.997629,6.947761,7.455664,6.871650,RZUF11_20220911_194500.WAV
234566,32.031155,32.447231,29.911245,30.538036,33.635494,34.014481,30.262552,30.493317,33.125942,26.479343,...,6.631141,7.389686,6.666187,6.783213,6.740025,6.848728,6.757042,6.959400,7.343256,RZUF11_20220911_194500.WAV
234567,29.657152,29.230629,30.523970,32.897121,32.165146,31.565163,31.108803,31.056589,30.828493,26.462791,...,7.224640,6.769463,6.860866,7.077959,6.688170,6.998895,6.503436,7.144662,7.056121,RZUF11_20220911_194500.WAV
234568,33.208477,35.691185,33.307129,29.323082,30.786558,29.438837,29.459129,29.440765,32.396168,26.464783,...,7.223698,7.188468,6.689512,6.920041,7.048953,6.827715,7.099158,7.051322,6.947255,RZUF11_20220911_194500.WAV
234569,29.813419,27.497622,30.227625,33.091118,28.860977,33.200691,32.206181,29.791512,31.413130,26.473598,...,6.864029,6.829520,7.115244,6.941751,7.099781,6.832318,6.932252,7.278535,7.387477,RZUF11_20220911_194500.WAV
234570,33.304573,34.962975,33.468311,31.832928,32.525333,32.582600,34.163605,32.543274,34.381058,24.959534,...,6.818027,6.720137,6.744768,6.830623,6.801695,6.739822,6.835235,6.844056,6.824719,RZUF11_20220911_200000.WAV
234571,33.401054,32.972130,33.740154,35.489105,33.646950,33.753338,32.501038,30.634752,32.654797,25.028490,...,6.809321,6.858594,6.861931,6.844831,6.859490,6.843732,6.814089,6.839853,6.807248,RZUF11_20220911_200000.WAV
234572,36.516533,33.248848,34.286911,33.582970,33.106743,34.489395,35.790222,34.834328,33.949867,24.959747,...,6.809158,6.816563,6.819107,6.785583,6.812307,6.802926,6.812215,6.822934,6.835976,RZUF11_20220911_200000.WAV
234573,34.865009,35.089550,32.467724,32.717308,32.748936,34.248707,31.980352,33.437935,35.005680,25.020201,...,6.807712,6.840740,6.841762,6.824104,6.821933,6.795653,6.816473,6.822402,6.790070,RZUF11_20220911_200000.WAV


### Autoencoders Features and Labels using independent segments

In [7]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_ae, labels_ae, test_size=0.2,random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)

y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

Accuracy: 0.7899145456781016
f1: 0.7188050972016402
recall 0.6935557365932045


### Autoencoders Features and Labels using 5 segments of the audio

In [8]:
X_batch = np.reshape(X_ae, (X_ae.shape[0]//5,5,X_ae.shape[1]))
y_batch = np.reshape(y_ae, (y_ae.shape[0]//5, 5))

NameError: name 'y_ae' is not defined

In [ ]:
y_3 == y_path[:,0]

In [ ]:
len(labels_ai), y_path[0].shape

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_batch, y_path, test_size=0.2,random_state=0)

X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[2])
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = y_train.reshape(y_train.shape[0]*y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1])

labels_train = []
for i in range(len(y_train)):
    labels_train.append(audios.loc[y_train[i], "cover"])
labels_test = []
for i in range(len(y_test)):
    labels_test.append(audios.loc[y_test[i], "cover"])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, labels_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(labels_test, y_pred_rf)
f1_score = metrics.f1_score(labels_test, y_pred_rf, average="macro")
recall = metrics.recall_score(labels_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

# Autoencoders Features and Labels using voting labels

In [ ]:
y_pred_rf2 = np.asarray(y_pred_rf)
y_pred_rf2 = np.reshape(y_pred_rf2,(y_pred_rf2.shape[0]//5,5))
y_test2 = np.asarray(labels_test)
y_test2 = np.reshape(y_test2,(y_test2.shape[0]//5,5))

def most_frequent(List):
    return max(set(List), key = List.count)

labels_test2 = []
labels_pred = []
for i in range(len(y_pred_rf2)):
    labels_pred.append(most_frequent(list(y_pred_rf2[i])))
    labels_test2.append(most_frequent(list(y_test2[i])))
accuracy_ae = metrics.accuracy_score(labels_test2, labels_pred)
f1_score_ae = metrics.f1_score(labels_test2, labels_pred, average="macro")
recall_ae = metrics.recall_score(labels_test2, labels_pred, average="macro")

print("Accuracy:", accuracy_ae)
print("f1:", f1_score_ae)
print("recall", recall_ae)